# <span style="color:cornflowerblue"> Gerald Jones</span>
# <span style="color:cornflowerblue"> Home Work 3: Capital Budgeting </span>
# <span style="color:cornflowerblue"> ISE522 Spg 22</span>

## Problem Description:
> * Suppose you are an investor, and you are considering investing in <span style="color:red"><b>three projects</b></span>.
> * It is possible to invest in a fractional amount of a project or the entire amount. For example, if we invest in 0.5 on project 3, then we have cash outflows of -$\$1$ million at time 0 and 0.5.) If you fully invest in a project, the realized cash flows, (in millions of dollars) will be as shown in the following Table.
> * Today we <span style="color:red"><b>have $\$2$ million in cash</b></span>. 
> * At each time period (0, 0.5, 1, 1.5, 2, and 2.5 years from today) we <span style="color:red"><b>may</b></span>, if desired, <span style="color:red"><b>borrow up to $\$2$ million at 3.5\% (per 6 months) interest and must be paid back in the next period. Leftover cash earns 3\% (per 6 months) interest</b></span>. For example, if after borrowing and investing at time 0 we have $\$$1 million we would receive $\$3$0,000 in interest at time 0.5 years.
> * Your <span style="color:red"><b>goal is to maximize cash on hand after accounting for time 3
cash flows</b></span>. Formulate an LP to accomplish this goal. Solve using
a solver of your choice.



# Notes:
### Parameters:
* Initial on hand cash
* set of investments
* Number of time steps to forcast
* Cash flows expected for each project in given time step
* Loan interest = 3.5% == .35*loan_amount_last_time_step
* Maximum allowable loan amount
* Cash on hand interest == 3% = .3*cash_on_hand_last_time_step
* three parts to a given time periods cash flow/cash on hand amount
    * cash on hand from previous time period 
        * i.e. current cash on hand from last time periods time flow
    * cash flow from investments
    * cash flow from loans borrowed
    * cash flow from loans repaided
### Variables and constraints:
* Decision: 
    * what proportion of given three choices to invest in
    * when to borrow
    * how much to borrow
* Decision constraints:
    * maximum investment amount (assumed $\leq$ 1 i.e. 100%)
    * maximum allowable borrowing amount
* Lending logic: 
    * may borrow >= 2M at 3.5% interest every size months(.5 years or every row/step)
        * must be paid back at next time period
        * pay back amount*interest + amount in next time step
    * we can not owe money at the end of the period i.e. cash flows can not be negative
        * if you will have a negative cash flow at the end of the period borrow enough so that it is not negative up to               maximum allowable amount
* Cash on hand logic:
    * cash on hand at time t(current) is based on cash at time t-1(last time step)
        * cash on hand at time t-1 gains interest of the set percentage rate leading to an additional c(*interest_rate) at           time t
        * thus cash currently held gains the interest from cash on hand in the previous step and is the sum of the amount             of cash on hand at the end of the last time step plus the interest for this amount


# objective:
> Maximize cash on hand to get maximimum cash returns arter the T time steps


IE 60? solving integer programs
# <span style="color:orange"><center><b>Module imports and data loading</b></center></span>

In [1]:
from _GUROBI_TOOLS_.GUROBI_MODEL_BUILDING_TOOLS import *
from _NOTE_BOOK_UTILS import *

data_df = pd.read_excel("HW3_Data.xlsx")
                        


<IPython.core.display.Javascript object>

# <span style="color:Green"><center><b>Data Display</b></center></span>

In [2]:
print("Data for Optimization Task:")
display(data_df)
print("columns:")
for v in data_df.columns:
    print(v)
    
    
Investment_lower_bound = data_df.sum(axis=1).min()

Investment_lower_bound

Data for Optimization Task:


,Years,project 1,project 2,project 3
0,0.0,-3.0,-2.0,-2.0
1,0.5,-1.0,-0.5,-2.0
2,1.0,1.8,1.5,-1.8
3,1.5,1.4,1.5,1.0
4,2.0,1.8,1.5,1.0
5,2.5,1.8,0.2,1.0
6,3.0,5.5,-1.0,6.0


columns:
Years
project 1
project 2
project 3


-7.0

# <center> <span style="color:blue"> Model Formulation</span> </center>

## Parameters:
>### **P** $\quad$   set of projects that can be invested in where p $\in$ <b>P</b>
>### **T** $\quad$   set of time steps for set time invterval = {0, .5, 1, 1.5, .... , 3}, such that t $\in$ **T**
>### **$G$** $\quad$      interest rate used to determine gains for cash on hand at some time t
>### **$L$** $\quad$      interest rate used to determine losses for money borrowed at some time  t
>### **$C_0$** $\quad$     initial cash on hand
>### <b>R$_{p,t}$</b>$\quad$   cash return for investing in project p at time t
>### <b>$\Lambda$</b>$\quad$   maximum allowable loan amount
## Variables:
### Investment decisions and returns/costs

##### Decision cash flows
>### <b>X$_{p}$</b>$\quad$   proportion of investment for project p
>### <b>A$_t$</b>$\quad$       amount borrowed at time t 

##### Investment/Interest rate cash flows 
>### <b>O$_t$</b>$\quad$       money owed at time t due to borrowed money at time t-1
>### <b>I$_{t}$</b>$\quad$     profits from cash returns on investments at time t
>### <b>N$_{t}$</b>$\quad$     1 if <b>I$_{t}$</b> < 0, 0 otherwise

##### Cash left over returns 
>### <b>C$_t$</b>$\quad$       cash on hand at the end of time period t
>### <b>C$_T$</b>$\quad$       cash on hand after T cash flow periods




    

    
## Equations and constraints:


>### <center><span style="font-size:30px;color:red"><b>Proportion of investments constraints</b></span></center>
# $$0 \leq \quad X_p \quad  \leq 1, \forall p$$
# $$I_t = \sum_{p=1}^{3} (X_{p} \cdot R_{p,t}), \forall t$$


>#### Negative investment costs can not exceed the maximum loan amount
# $$ abs(I_t) \geq  \Lambda$$
# $$N_t  == 0, \implies \text{ } I_t + A_t + C_{t-1} + C_{t-1} \cdot G - O_t \geq 0 $$

>### <center><span style="font-size:30px;color:red"><b>Money borrowed at time t constraints</b></span></center>
# $$A_t \quad  \leq \Lambda$$

>### <center><span style="font-size:30px;color:red"><b>Money owed at time t for loans in t-1:</b></span></center>
# $$O_t =   0, t=0$$
# $$O_t \quad =   A_{t-1} + (A_{t-1} \cdot L), \forall t > 0$$

>### <center><span style="font-size:30px;color:red"><b>Cash on hand/flow at t and after T time steps:</b></span></center>
# $$C_t > 0, \forall \text{ t}$$ 
# $$C_t = K + A_t + I_t, t = 0$$ 
# $$C_t = I_t + A_t - O_t + C_{t-1} + C_{t-1} \cdot G, \forall t > 0$$  
# $$C_T =\sum_{t=0}^{T}(C_t)$$ 


>### <center><span style="font-size:30px;color:red"><b>When to borrow constraint</b></span></center>
>#### Have to borrow enough so that the cash flow at time t will not be negative
# $$A_t \geq   -I_t - (C_{t-1} \cdot L) - C_{t-1} + O_t, \forall t$$













    
## Objective:
> ## Goal: maximize cash on hand after forcast period (3 years at 6 month intervals):
    
## $$\max( (C_{T}))$$

In [3]:
col_name = data_df.columns.to_list()[3]
col_num = 2
for idx in data_df.index:
    print("{}-{}: {}".format(idx, col_name, data_df.iloc[idx, 2]))

0-project 3: -2.0
1-project 3: -0.5
2-project 3: 1.5
3-project 3: 1.5
4-project 3: 1.5
5-project 3: 0.2
6-project 3: -1.0


In [4]:
# # cash sum of cash on hand 
# def generate_time_step_objective(model, Cs, return_rate):
#     expression = None
#     for c in range(len(Cs)):
#         if c == 0:
#             expression = Cs[c] * return_rate + Cs[c]
#         else:
#             expression += Cs[c] * return_rate + Cs[c]
#     return expression

# def generate_onhand_constraints(model, Cs, Ps, As, Ots, initial):
#     for c in range(len(Cs)):
#         if c == 0:
#             model.addConstr(Cs[c] == initial + Ps[0,c] - Ots[0,c] + As[0,c])
#         else:
#             model.addConstr(Cs[c] == Cs[c-1] + Ps[0,c] - Ots[0,c] + As[0,c])

            
# def generate_investment_profit_constraints(model, Ps, Isp, ):          
#     pass
  


# def generate_profit_by_project_variable_constraint(model, Xs, Ps, df, rate_col_format, col_base=1):
#     # for each time step t
#     for t in range(len(Ps)):
#         # for each project X[p]
#         expression = 0
#         for p in range(len(Xs)):
#             project_number = p+col_base
#             print("{}, {}, {}".format(t, rate_col_format.format(project_number), df.loc[t, rate_col_format.format(project_number)]))
#             # for each time step t do : X[p] + R[p, t]
# #             if p == 0:
# #                 expression = Xs[p] * df.loc[t, rate_col_format.format(project_number)]
# #             else:
# #             expression += Xs[p] * df.loc[t, rate_col_format.format(project_number)]
        
# #         # after all projects have been included for this time step
# #         # add the constraint for the profit at this time step
# #         model.addConstr(Ps[t] == expression)
#     return 


# def generate_multi_dict_from_df(df, key_col, val_col=["", ""]):
#     ret_dict = {}
#     for idx in df.index:
#         print(df.loc[idx, key_col], " :", df.loc[idx, val_col].tolist())
#         ret_dict[df.loc[idx, key_col]] = df.loc[idx, val_col].tolist()
#     return ret_dict   
        
# def add_sequential_operation(model, Xws, Ows, Dws, Hws, initval):
#     for i in range(0, len(Xws)):
#         # use the given initial value for the amount at the end of the first week
#         if i == 0:
#             print("initval ", initval)
#             model.addConstr(Xws[i]*Ows[i] + initval - Dws[i] == Hws[i])
#         else:
#             model.addConstr(Xws[i]*Ows[i] + Hws[i-1] - Dws[i] == Hws[i])
            
            
# def add_product_GEQ(model, Xws, Ows, minval):
#     for i in range(0, len(Xws)):
#             model.addConstr(Xws[i]*Ows[i] == Xws[i])

            
# def add_value_EQconstraints(model, value,  Vl, idx):
#     model.addConstr(Vl[idx] == value)
        
# def add_value_GEQconstraints(model, value,  Vl, idx):
#     model.addConstr(Vl[idx] >= value)

# def add_value_GEQconstraintS(model, value,  Vl, start=0):
#     for idx in range(start, len(Vl)):
#         model.addConstr(Vl[idx] >= value)
        
# def add_value_LEQconstraints(model, value,  Vl, idx):
#     model.addConstr(Vl[idx] <= value)

    
    
# def add_value_EQconstraintsDF(model, df, Vl, col="Minimum inventory requirement"):
#     for i in range(len(Vl)):
#         add_value_EQconstraints(model, df.loc[i, col],  Vl, i)

        
# def add_value_GEQconstraintsDF(model, df, Vl, col="Minimum inventory requirement"):
#     for i in range(len(Vl)):
#         add_value_GEQconstraints(model, df.loc[i, col],  Vl, i)
        
# def add_value_LEQconstraintsDF(model, df, Vl, col="Minimum inventory requirement"):
#     for i in range(len(Vl)):
#         add_value_LEQconstraints(model, df.loc[i, col],  Vl, i)        

        
        
        
# generate_multi_dict_from_df(data_df, key_col="Years", val_col="project 1")        

# <center>Constraint methods</center>

In [5]:
def generate_money_owed_constraints(model, Os, As, l):
    for t in range(len(Os)):
        if t == 0:
            model.addConstr(Os[t] == 0)
        else:
            model.addConstr(Os[t] == As[t-1] + As[t-1]*l)
    return 




def generate_profit_by_project_variable_constraint(model, Xs, Ps, df, rate_col_format, col_base=1):
    # for each time step t
    for t in range(len(Ps)):
        # for each project X[p]
        expression = 0
        for p in range(len(Xs)):
            project_number = p+col_base
            print("t{}, p{}, {}".format(t, 
                                      rate_col_format.format(project_number), 
                                      df.loc[t, rate_col_format.format(project_number)]))
            # for each time step t do : X[p] * R[p, t]
            if p == 0:
                expression = (Xs[p] * df.loc[t, rate_col_format.format(project_number)]) 
            else:
                expression += (Xs[p] * df.loc[t, rate_col_format.format(project_number)])
        
        # after all projects have been included for this time step
        # add the constraint for the profit at this time step
        model.addConstr(Ps[t] == expression)
    return 

# <center>Solve</center>

In [6]:

try:
    #########################################################################################
    ################################## Parameters set up ####################################
    #########################################################################################
    # get the number of time steps 
    T = len(data_df)
    
    # interest on mony left in hand
    G = .3
    
    # interest rate on borrowed money
    L = .35
    
    
    # Initial cash on hand
    K = 2
    
    # lambda: maximum loan amount
    Lmbda = 2
    
    # initial investment in millions
    
    print("Parameters:\n\t\t T: {}, G: {}, L:{}, Lambda: {}, Initial Capital: {}".format(T, G, L, Lmbda, K))
    #########################################################################################
    ################################## Variables set up #####################################
    #########################################################################################
    # get the investment cashflows for the years for each given project
#     cflw = gp.multidict(generate_multi_dict_from_df(data_df, key_col="Years", val_col=["project 1", "project 2", "project 3"]) )
#     print(yrs)
#     print(cflw)
    
    # Create a new model
    m = gp.Model("CapitalBudgeting")
    
#     Ze = m.addVar(vtype=GRB.BINARY, name="MIN",ub=0)
#     m.addConstr(Ze == 0)
    # ##################################################################################################
    # ##################################### Investment Based variables #######################################
    # ##################################################################################################
    # add Investment portions for project variables
    Xp = m.addVars(3, vtype=GRB.CONTINUOUS, name="X", lb=0, ub=1)
   
    # ##################################################################################################
    # ##################### Cash flow/on hand at end of time t variables ###############################
    # ##################################################################################################
    # money on hand variables at time t
    Ct = m.addVars(T, vtype=GRB.CONTINUOUS, name="C")


    # add rate of cash on hand interest variables for project p in time t
#     Rpt = m.addVars(3, T, vtype=GRB.CONTINUOUS, name="R")
    
    # add profits from investments(cash on hand from last time period profits) variables
    #     form P[t] = sum:p=1->3:=X[p]*R[p,t]  == X[p]*df[t,p]
    # use lower bound to ensure can only invest in what we can borrow to get out of
#     It = m.addVars(T, vtype=GRB.CONTINUOUS, name="I", lb=[-Lmbda*Ct[t] for t in range(T)])
    It = m.addVars(T, vtype=GRB.CONTINUOUS, name="I", lb=Investment_lower_bound)
    
    
    
    # ##################################################################################################
    # ##################################### Loan Based variables #######################################
    # ##################################################################################################
    
    # add amount loan variables for some proportion of the loan for a project
    At = m.addVars(T, vtype=GRB.CONTINUOUS, name="A", lb=0, ub=Lmbda)
 
    # money owed from loans
    Ot = m.addVars(T, vtype=GRB.CONTINUOUS, name="O")
    
    
    
    #########################################################################################
    ################################## Objective set up #####################################
    #########################################################################################
    #                     Need to maximize the cash on hand at the end of T cash flows represented by the last
    #                     Ct variable
    m.setObjective(Ct[len(Ct)-1], GRB.MAXIMIZE) 
    #########################################################################################
    ################################## Constraint set up #####################################
    #########################################################################################
     ##                                     Profits from investments at time t:
    #     generate_profit_by_project_variable_constraint(model, Xs, Ps, Cs, df, rate_col_format, col_base=1, ):
#     generate_profit_by_project_variable_constraint(m, Ip, Pt, data_df, rate_col_format="project {}", col_base=1)
# generate_profit_by_project_variable_constraint(model, Xs, Ps, df, rate_col_format, col_base=1)
    generate_profit_by_project_variable_constraint(m, Xp, It, data_df, rate_col_format="project {}", col_base=1)
#     Xs[p] * df.loc[t, rate_col_format.format(project_number)]
#     p+col_base
#     rate_col_format = "project {}" 
#     m.addConstrs(Pt[t] = Xp[p] * df.loc[t, "project {}".format(p+1)]  for t in range(len(Pt))
#                                                                            for p in range(len(Xp)))
    
    ##           Money owed at time t for loans in t-1:
    # add constraint for loans not occuring if the previous half year had one
    ##           Money borrowed at time t constraints
    generate_money_owed_constraints(m, Ot, At, L)
    
    
    
    ##           Cash on hand at t:
#     # cash on hand constraint/equation
#     def add_cash_on_hand(model, Cs, Is, As, Os, k=K, g=G):
#         for i in range(len(Cs)):
#             if i == 0:
#                 model.addConstr(Cs[i] == k + As[i], Is[i])
#             else:
#                 model.addConstr(Cs[i] == Is[i] + As[i] - Os[i] + Cs[t-1] +  Cs[t-1]*g )    
    
#     print(Ct)
#     add_cash_on_hand(m, Ct, Pt, At, Ot, K)
    m.addConstr(Ct[0] == K + At[0] + It[0])
    m.addConstrs(Ct[t] == Ct[t-1] + Ct[t-1]*G + At[t] + It[t] - Ot[t] for t in range(1, len(Ct)))    
#     #                                             add the need to borrow constraint
#     def generate_need_to_borrow(model, Ps, As, Is, Os, g):
#         for t in range(len(Ps)):
            
#             model.addConstr(A[t] >= Os[t] - Ps[t] - C[t-1] - C[t-1]*g )
#         return 
#     generate_need_to_borrow(m, Pt, At, Bt, Ot)
    m.addConstr(At[0] >= Ot[0] - It[0] - Ct[0])
    m.addConstrs(At[t] >= Ot[t] - It[t] - Ct[t-1] - Ct[t-1]*G for t in range(1, len(At)))
    
    
    ##           
    ##           
    #########################################################################################
    ################################## SOLVE:OPTIMIZE ######################################
    #########################################################################################
    
    m.optimize()
    displayDecisionVars(m, end_sentinel="6")
    
    print("\n-------------Does it make sense?----------------------")  
    print('Obj-Profit after 10 months: ${:,.2f}'.format(m.ObjVal*1000000))
    
    
# catch some math errors
except gp.GurobiError as e:
    print('Error code ' + str(e.errno) + ': ' + str(e))

except AttributeError:
    print('Encountered an attribute error')

Parameters:
		 T: 7, G: 0.3, L:0.35, Lambda: 2, Initial Capital: 2
Restricted license - for non-production use only - expires 2023-10-25
{0: <gurobi.Var *Awaiting Model Update*>, 1: <gurobi.Var *Awaiting Model Update*>, 2: <gurobi.Var *Awaiting Model Update*>, 3: <gurobi.Var *Awaiting Model Update*>, 4: <gurobi.Var *Awaiting Model Update*>, 5: <gurobi.Var *Awaiting Model Update*>, 6: <gurobi.Var *Awaiting Model Update*>}
t0, pproject 1, -3.0
t0, pproject 2, -2.0
t0, pproject 3, -2.0
t1, pproject 1, -1.0
t1, pproject 2, -0.5
t1, pproject 3, -2.0
t2, pproject 1, 1.8
t2, pproject 2, 1.5
t2, pproject 3, -1.8
t3, pproject 1, 1.4
t3, pproject 2, 1.5
t3, pproject 3, 1.0
t4, pproject 1, 1.8
t4, pproject 2, 1.5
t4, pproject 3, 1.0
t5, pproject 1, 1.8
t5, pproject 2, 0.2
t5, pproject 3, 1.0
t6, pproject 1, 5.5
t6, pproject 2, -1.0
t6, pproject 3, 6.0
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a mod

# Save Notebook:
## Run the cell below to save the current notebook as a pdf in the same directory

In [ ]:
to_PDF()